In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/memory-test-on-drugged-islanders-data/Islander_data.csv


In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')


In [32]:
data = pd.read_csv('/kaggle/input/memory-test-on-drugged-islanders-data/Islander_data.csv')

In [33]:
data

,first_name,last_name,age,Happy_Sad_group,Dosage,Drug,Mem_Score_Before,Mem_Score_After,Diff
0,Bastian,Carrasco,25,H,1,A,63.5,61.2,-2.3
1,Evan,Carrasco,52,S,1,A,41.6,40.7,-0.9
2,Florencia,Carrasco,29,H,1,A,59.7,55.1,-4.6
3,Holly,Carrasco,50,S,1,A,51.7,51.2,-0.5
4,Justin,Carrasco,52,H,1,A,47.0,47.1,0.1
...,...,...,...,...,...,...,...,...,...
193,Jacob,Novak,52,H,3,T,71.3,74.3,3.0
194,Teo,Steiner,41,S,3,T,72.5,70.4,-2.1
195,Alexander,Takahashi,54,S,3,T,30.8,33.1,2.3
196,Alexandere,Takahashi,40,H,3,T,53.6,53.8,0.2


In [34]:
def onehot_encode(df, column):
    df = df.copy()
    
    dummies = pd.get_dummies(df[column], prefix = column)
    if len(df[column].unique()) == 2:
        dummies = dummies.drop(dummies.columns[0], axis = 1)
        
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
    
    return df

In [35]:
def preprocess_inputs(df):
    df = df.copy()
    
    ## One Hot encode categorical features:
    for column in ['first_name', 'last_name', 'Happy_Sad_group']:
        df = onehot_encode(df, column = column)
    
    ## Split the data 
    y = df['Drug']
    X = df.drop('Drug', axis = 1)
    
    ## Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state = 1)
    
    ## Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [36]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [37]:
X_train

,age,Dosage,Mem_Score_Before,Mem_Score_After,Diff,first_name_Aaron,first_name_Adam,first_name_Ai,first_name_Akane,first_name_Akira,...,last_name_Lopez,last_name_McCarthy,last_name_Morin,last_name_Novak,last_name_Price,last_name_Rodriguez,last_name_Steiner,last_name_Summers,last_name_Takahashi,Happy_Sad_group_S
124,-0.302247,1.206716,0.249183,-0.151850,-0.594735,0.0,-0.121268,0.0,-0.121268,0.0,...,2.761340,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,1.0
97,0.909251,0.025675,1.221038,1.038471,-0.092208,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,1.0
42,1.428464,0.025675,0.438505,-0.684661,-1.707473,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,3.785939,-0.336011,1.0
17,-1.167603,-1.155366,0.413262,0.698379,0.518003,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,11.704700,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,-1.0
5,-0.215712,-1.155366,0.564720,-0.117841,-0.989577,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,-0.215712,-1.155366,-0.665875,-0.752679,-0.244761,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,1.0
137,1.947677,-1.155366,2.432701,2.370498,0.293661,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,-1.0
72,1.082322,-1.155366,0.552099,0.069210,-0.675498,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,1.0
140,-0.475319,-1.155366,-0.533349,-1.115444,-1.007525,0.0,-0.121268,0.0,-0.121268,0.0,...,-0.362143,-0.23116,-0.085436,-0.085436,0.0,-0.085436,-0.264135,-0.264135,-0.336011,1.0


In [39]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
#     "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                              LightGBM trained.
                              CatBoost trained.


In [40]:
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))

                   Logistic Regression: 41.67%
                   K-Nearest Neighbors: 36.67%
                         Decision Tree: 45.00%
Support Vector Machine (Linear Kernel): 40.00%
   Support Vector Machine (RBF Kernel): 43.33%
                        Neural Network: 38.33%
                         Random Forest: 43.33%
                     Gradient Boosting: 50.00%
                              LightGBM: 38.33%
                              CatBoost: 48.33%
